In [4]:
import requests
import pandas as pd

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()

df = pd.DataFrame(data)

df.shape

(230, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Intitulé du poste    230 non-null    object
 1   Date de publication  230 non-null    object
 2   lieu                 230 non-null    object
 3   competences          230 non-null    object
 4   Type de poste        230 non-null    object
dtypes: object(5)
memory usage: 9.1+ KB


In [6]:
df.describe()

,Intitulé du poste,Date de publication,lieu,competences,Type de poste
count,230,230,230,230,230
unique,203,32,72,203,79
top,[DEVELOPPEUR AGILE-(H/F)\n],\npostée il y a 1 mois\n,[\nParis\n],"[\nagile\n, \nsql\n, \nsupport\n, \npython\n, ...","[\n, \n, Société générale, \n, Client final, \..."
freq,3,24,38,4,35


In [8]:
df.columns

Index(['Intitulé du poste', 'Date de publication', 'lieu', 'competences',
       'Type de poste'],
      dtype='object')

In [10]:
df.count()

Intitulé du poste      230
Date de publication    230
lieu                   230
competences            230
Type de poste          230
dtype: int64

In [66]:
import requests
import pandas as pd
import re

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

# Extracting numeric value from the lieu column and storing in the 'salaire' column
df['salaire'] = df['lieu'].apply(lambda x: [i for i in x if i.find("€")!=-1])

# Remove newline characters and format the data
df = df.replace('\n','',regex=True)
df['Intitulé du poste'] = df['Intitulé du poste'].apply(lambda x: [i.strip() for i in x])
df['Date de publication'] = df['Date de publication'].apply(lambda x: x.strip().replace("postée ", "").replace("il y a ", ""))
df['lieu'] = df['lieu'].apply(lambda x: [i.strip() for i in x])
df['competences'] = df['competences'].apply(lambda x: [i.strip() for i in x])
df['Type de poste'] = df['Type de poste'].apply(lambda x: [i.strip() for i in x])
df2 = df
df

,Intitulé du poste,Date de publication,lieu,competences,Type de poste,salaire
0,[Data Analyst - Publicité],21 jours,[Paris],"[sql, t-sql, outils]","[, , Sept Lieues SAS, , Cabinet de recrutement...",[]
1,[Data analyst - H/F],1 mois,"[Paris, La Défense]","[support, outils, agile, design, big data]","[, , EDF France, , Client final, , , CDI - tem...",[]
2,[Customer Data Analyst (H/F)],27 jours,[Paris],"[crm, data quality, sql, sas, python]","[, , Hermès, , Client final, , , CDI - temps p...",[]
3,[Master Data Analyst H/F],21 jours,[Moussy-le-Vieux],"[si, support, sap]","[, , TEREOS, , Client final, , , CDI - temps p...",[]
4,[Portfolio Data Analyst H/F],3 jours,[France],"[outils, support, decisionnel, sql, test]","[, , Safran-Group, , Client final, , , CDI - t...",[]
...,...,...,...,...,...,...
225,[SAP Fiori Developer (H/F)],27 jours,[Saint-Ouen],"[sap, abap, progiciel, système, support]","[, , Groupe ENGIE, , Client final, , , CDI - t...",[]
226,[ENGIE Digital - Architecte Données (H/F)],3 jours,[Paris],"[digital, outils, securité, sécurité, t-sql]","[, , Groupe ENGIE, , Client final, , , CDI - t...",[]
227,[Quality Assurance - Lead QA - Lead Acceptance...,2 heures,[Boulogne-Billancourt],"[test, serveur, jquery, mobile, tdd]","[, , Francaise des jeux, , Client final, , , C...",[]
228,[DATA SCIENTIST],16 jours,"[NEUILLY SUR SEINE, 38.000,00 € - 50.000,00 € ...","[nosql, unix, java, machine learning, linux]","[, , BEEZEN, , ESN, , , CDI - temps plein]","[\n38.000,00 € - 50.000,00 € / an\n]"


In [67]:
df2['salaire'] = df2['salaire'].apply(lambda x: x[0].replace('\n','') if x else '')
df3 = df2
df2.head(10)

,Intitulé du poste,Date de publication,lieu,competences,Type de poste,salaire
0,[Data Analyst - Publicité],21 jours,[Paris],"[sql, t-sql, outils]","[, , Sept Lieues SAS, , Cabinet de recrutement...",
1,[Data analyst - H/F],1 mois,"[Paris, La Défense]","[support, outils, agile, design, big data]","[, , EDF France, , Client final, , , CDI - tem...",
2,[Customer Data Analyst (H/F)],27 jours,[Paris],"[crm, data quality, sql, sas, python]","[, , Hermès, , Client final, , , CDI - temps p...",
3,[Master Data Analyst H/F],21 jours,[Moussy-le-Vieux],"[si, support, sap]","[, , TEREOS, , Client final, , , CDI - temps p...",
4,[Portfolio Data Analyst H/F],3 jours,[France],"[outils, support, decisionnel, sql, test]","[, , Safran-Group, , Client final, , , CDI - t...",
5,[Data Analyst BO/Qlick Sense (h/f)],15 jours,"[Maisons Alfort, 45.000,00 € - 50.000,00 € / an]","[support, sql, data management, business objec...","[, , FED IT, , Cabinet de recrutement, , , CDD...","45.000,00 € - 50.000,00 € / an"
6,[Product Data Analyst - BI - H/F],28 jours,[Paris],"[cloud, paas, qlik, iaas, support]","[, , Cegid, , Client final, , , CDI - temps pl...",
7,[Data Analyst / Chef de Projet BI (h/f)],1 mois,"[PARIS, 60.000,00 € - 62.000,00 € / an]","[data management, business intelligence, bases...","[, , FED IT, , Cabinet de recrutement, , , CDI...","60.000,00 € - 62.000,00 € / an"
8,[Business analyst senior DATA CRM - Paris H/F],12 heures,[Paris],"[crm, agile, decisionnel, réseaux, réseau]","[, , Credit Agricole, , Client final, , , CDI ...",
9,[Data scientist / Analyst and software develop...,13 jours,[Rueil-Malmaison],"[big data, scrum, intelligence artificielle, e...","[, , Vinci, , Client final, , , Stage]",
